<a href="https://colab.research.google.com/github/alissonjcjk/Projeto-A.M/blob/main/Projeto_AM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Projeto A.M

/content/Projeto-A.M


# Definição de blibiotecas

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import  train_test_split
from sklearn.metrics import ConfusionMatrixDisplay ,mean_absolute_error, accuracy_score,mean_squared_log_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Carregar corretamente o arquivo com separação por espaços
data = pd.read_csv("/content/dataset/yeast.data", sep='\s+', header=None)
data.columns = [
    'name',  # Nome da sequência: identificador da proteína no banco SWISS-PROT
    'mcg',   # McGeoch's method: método de reconhecimento de sequência sinal (score numérico)
    'gvh',   # von Heijne's method: outro método de reconhecimento de sequência sinal (score numérico)
    'alm',   # ALOM score: pontuação do programa ALOM para predição de regiões transmembrana
    'mit',   # Mitochondrial discriminant: score da análise discriminante do conteúdo de aminoácidos na região N-terminal (20 resíduos) para distinguir proteínas mitocondriais de não-mitocondriais
    'erl',   # HDEL presence: atributo binário (0 ou 1) indicando a presença do motivo "HDEL", sinal de retenção no lúmen do retículo endoplasmático
    'pox',   # Peroxisomal targeting signal: score indicando a presença de sinal de direcionamento para peroxissomos no extremo C-terminal
    'vac',   # Vacuolar discriminant: score da análise discriminante do conteúdo de aminoácidos para distinguir proteínas vacuolares de extracelulares
    'nuc',   # Nuclear localization signal: score da análise discriminante de sinais de localização nuclear (proteínas nucleares vs não nucleares)
    'class'  # Classe-alvo: localização celular da proteína (ex: CYT, NUC, MIT, etc.) — variável categórica não numérica
]
data

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4049883588.py:2: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv("/content/dataset/yeast.data", sep='\s+', header=None)


,name,mcg,gvh,alm,mit,erl,pox,vac,nuc,class
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT
...,...,...,...,...,...,...,...,...,...,...
1479,YUR1_YEAST,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22,ME2
1480,ZIP1_YEAST,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47,NUC
1481,ZNRP_YEAST,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22,ME2
1482,ZUO1_YEAST,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39,NUC


In [4]:
# Quantidades de exemplos para cada classe:
data['class'].value_counts()

,count
class,
CYT,463
NUC,429
MIT,244
ME3,163
ME2,51
ME1,44
EXC,35
VAC,30
POX,20


# Codicação das classes

In [5]:
from sklearn.preprocessing import LabelEncoder

# Supondo que 'data' já está carregado e com colunas nomeadas
le = LabelEncoder()

# Cria uma nova coluna 'class_encoded' com valores numéricos (0, 1, 2, ...)
data['class_encoded'] = le.fit_transform(data['class'])
data

,name,mcg,gvh,alm,mit,erl,pox,vac,nuc,class,class_encoded
0,ADT1_YEAST,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT,6
1,ADT2_YEAST,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT,6
2,ADT3_YEAST,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT,6
3,AAR2_YEAST,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC,7
4,AATM_YEAST,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT,6
...,...,...,...,...,...,...,...,...,...,...,...
1479,YUR1_YEAST,0.81,0.62,0.43,0.17,0.5,0.0,0.53,0.22,ME2,4
1480,ZIP1_YEAST,0.47,0.43,0.61,0.40,0.5,0.0,0.48,0.47,NUC,7
1481,ZNRP_YEAST,0.67,0.57,0.36,0.19,0.5,0.0,0.56,0.22,ME2,4
1482,ZUO1_YEAST,0.43,0.40,0.60,0.16,0.5,0.0,0.53,0.39,NUC,7


# Definição de Features

In [6]:
# Definir features (X) e alvo (y)
X = data.drop(['name', 'class_encoded','class'], axis=1)  # Remove colunas não preditivas
y = data['class_encoded']                         # Variável alvo: localização celular

In [7]:
# Dividir em 70% treino e 30% teste, com estratificação pela classe
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,          # 30% para teste
    stratify=y,              # Mantém a proporção de cada classe em treino e teste
    random_state=42          # Reprodutibilidade
)

In [8]:
train_X, val_X, train_y, val_y = train_test_split( X_train, y_train, random_state = 0)

In [9]:
train_X

,mcg,gvh,alm,mit,erl,pox,vac,nuc
649,0.61,0.59,0.47,0.49,0.5,0.0,0.54,0.22
358,0.27,0.51,0.47,0.35,0.5,0.0,0.49,0.28
1174,0.51,0.48,0.51,0.26,0.5,0.0,0.51,0.27
77,0.50,0.54,0.53,0.54,0.5,0.0,0.51,0.27
467,0.43,0.51,0.60,0.25,0.5,0.0,0.55,0.22
...,...,...,...,...,...,...,...,...
692,0.33,0.48,0.55,0.16,0.5,0.0,0.53,0.75
117,0.47,0.47,0.57,0.48,0.5,0.0,0.48,0.22
1421,0.63,0.65,0.42,0.18,0.5,0.0,0.54,0.27
403,0.53,0.43,0.47,0.15,0.5,0.0,0.58,0.22


In [11]:
train_y

,class_encoded
649,6
358,7
1174,0
77,6
467,0
...,...
692,7
117,6
1421,5
403,0


# Treinamentos:

## Árvores de decisão

## Naive Bayes (Bayesiano ingênuo) GaussianNB

## Regressão logística

## k-vizinhos mais próximos (k-NN)

## Random Forest

# Resultados